Test to pull data (extract) from yahoofinancials API (/v6/finance/quote) and put data into a dataframe.
Uses Henry's basic subscription (100 daily calls) API key obtained from yahoofinancials web site
Tested with one symbol ("DIS") then with multiple symbols ("AAPL,BTC-USD,DIS,AMZN").  Successfully able to get the data for all symbols and bring them into a new DataFrame.

Excluded: open our db and insert the DataFrame data.  Initial approach is to take all the data columns returned and insert it into the db.

In [1]:
# from requests import get
import pandas as pd

In [2]:
##  Initial attempt to loop through the stock symbols; FAILS because we need to include header and api key.
##  This is attempted correctly in the subsequent cells 

# stocks=['AAPL', 'DIS', 'UBER']
# url='https://yfapi.net/v6/finance/quote?region=US&lang=en&symbols='
# https://yfapi.net/v6/finance/quote?region=US&lang=en&symbols=AAPL%2CBTC-USD%2CEURUSD%3DX

# for s in stocks:
#     url_s=url + s
#     response = get(url_s)
#     print(response.json)

In [3]:
## Successful query of /v6/finance/quote API
##  Need to have your own config.py file in same folder so you can import your api key
import requests
import pprint
from config import API_KEY

url = "https://rest.yahoofinanceapi.com/v6/finance/quote"

## Example (Warning: no space between symbols):
querystring = {"symbols":"AAPL,BTC-USD,DIS,AMZN"}
# querystring = {"symbols" : "DIS"}

# use API_KEY.  Send it in the header of the request
headers = {
    'x-api-key': API_KEY
    }

In [3]:
## Keep the actual request in a separate cell so that we only run this whenever we want
##  to get more data from the API site.  We avoid reaching our daily request limit too soon.
response = requests.request("GET", url, headers=headers, params=querystring).json()

# What does the response look like?  We will have to use indexing to get to the 'result' level
pprint.pprint(response)

{'quoteResponse': {'error': None,
                   'result': [{'ask': 142.52,
                               'askSize': 9,
                               'averageAnalystRating': '1.9 - Buy',
                               'averageDailyVolume10Day': 105565183,
                               'averageDailyVolume3Month': 79054577,
                               'bid': 142.53,
                               'bidSize': 9,
                               'bookValue': 3.882,
                               'currency': 'USD',
                               'displayName': 'Apple',
                               'dividendDate': 1628726400,
                               'earningsTimestamp': 1627403400,
                               'earningsTimestampEnd': 1635768000,
                               'earningsTimestampStart': 1635332340,
                               'epsCurrentYear': 5.59,
                               'epsForward': 5.71,
                               'epsTrailingTwelveMonths':

In [1]:
## How many records did we get?
len(response['quoteResponse']['result'])

NameError: name 'response' is not defined

In [9]:
## Parse through the resulting dictionary tree to get at the 'results'.  Look at only the first
##  result in order to see what columns we received.
response['quoteResponse']['result'][0]

{'language': 'en-US',
 'region': 'US',
 'quoteType': 'EQUITY',
 'quoteSourceName': 'Nasdaq Real Time Price',
 'triggerable': True,
 'currency': 'USD',
 'averageDailyVolume10Day': 105565183,
 'fiftyTwoWeekLowChange': 39.129997,
 'fiftyTwoWeekLowChangePercent': 0.37953442,
 'fiftyTwoWeekRange': '103.1 - 157.26',
 'fiftyTwoWeekHighChange': -15.029999,
 'fiftyTwoWeekHighChangePercent': -0.09557421,
 'fiftyTwoWeekLow': 103.1,
 'fiftyTwoWeekHigh': 157.26,
 'dividendDate': 1628726400,
 'earningsTimestamp': 1627403400,
 'earningsTimestampStart': 1635332340,
 'earningsTimestampEnd': 1635768000,
 'trailingAnnualDividendRate': 0.835,
 'trailingPE': 27.844559,
 'trailingAnnualDividendYield': 0.0057168286,
 'epsTrailingTwelveMonths': 5.108,
 'epsForward': 5.71,
 'epsCurrentYear': 5.59,
 'priceEpsCurrentYear': 25.443647,
 'sharesOutstanding': 16530199552,
 'bookValue': 3.882,
 'fiftyDayAverage': 149.31647,
 'fiftyDayAverageChange': -7.0864716,
 'fiftyDayAverageChangePercent': -0.047459412,
 'twoHund

In [10]:
# Create a new DataFrame from all the results in the dictionary
response_df = pd.DataFrame(response['quoteResponse']['result'])
response_df.head()

,language,region,quoteType,quoteSourceName,triggerable,currency,averageDailyVolume10Day,fiftyTwoWeekLowChange,fiftyTwoWeekLowChangePercent,fiftyTwoWeekRange,...,displayName,symbol,startDate,coinImageUrl,circulatingSupply,lastMarket,volume24Hr,volumeAllCurrencies,fromCurrency,toCurrency
0,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,105565183,39.129997,0.379534,103.1 - 157.26,...,Apple,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,en-US,US,CRYPTOCURRENCY,CoinMarketCap,True,USD,32767843516,33642.310000,3.302872,10185.774 - 64863.098,...,NaN,BTC-USD,1.367104e+09,https://s.yimg.com/uc/fin/img/reports-thumbnai...,18821492.0,CoinMarketCap,4.081262e+10,4.081262e+10,BTC,USD=X
2,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,6703050,60.799995,0.518639,117.23 - 203.02,...,NaN,DIS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,2842400,468.745120,0.163269,2871.0 - 3773.08,...,Amazon.com,AMZN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
